# Data Modelling

Data modeling is the process of creating a data model for the data to be stored in a database. This process involves defining and structuring the data elements and their relationships to one another. In the context of Databricks, data modeling is crucial for setting up new tables.


In [0]:
%sql
-- USERS TABLE
CREATE TABLE IF NOT EXISTS users (
    user_id STRING NOT NULL,
    default_city STRING,
    default_state STRING,
    CONSTRAINT pk_users PRIMARY KEY (user_id)
) USING DELTA;

-- DEVICES TABLE
CREATE TABLE IF NOT EXISTS devices (
    device_id STRING NOT NULL,
    device_os STRING,
    CONSTRAINT pk_devices PRIMARY KEY (device_id)
) USING DELTA;

-- RECIPIENTS TABLE (Merchants)
CREATE TABLE IF NOT EXISTS recipients (
    merchant_id STRING NOT NULL,
    merchant_category STRING,
    CONSTRAINT pk_recipients PRIMARY KEY (merchant_id)
) USING DELTA;

-- TRANSACTIONS TABLE
CREATE TABLE IF NOT EXISTS transactions (
    transaction_id STRING NOT NULL,
    transaction_date DATE,
    transaction_time STRING,
    user_id STRING NOT NULL,
    merchant_id STRING NOT NULL,
    device_id STRING NOT NULL,
    upi_channel STRING,
    transaction_city STRING,
    transaction_state STRING,
    ip_address STRING,
    transaction_status STRING,
    amount FLOAT,
    transaction_amount_deviation FLOAT,
    fraud INT,
    CONSTRAINT pk_transactions PRIMARY KEY (transaction_id),
    CONSTRAINT fk_txn_user FOREIGN KEY (user_id) REFERENCES users(user_id),
    CONSTRAINT fk_txn_device FOREIGN KEY (device_id) REFERENCES devices(device_id),
    CONSTRAINT fk_txn_merchant FOREIGN KEY (merchant_id) REFERENCES recipients(merchant_id)
) USING DELTA;




####Script for Data Generation

In [0]:
from faker import Faker
from datetime import datetime, timedelta
import random
from collections import defaultdict
from pyspark.sql.types import (
    StructType, StructField, StringType, FloatType, DateType, IntegerType
)
from pyspark.sql.functions import col, hour, unix_timestamp, when, date_format

# For demonstration, using a static list:
city_state_dict = {
    "Mumbai": "Maharashtra",
    "Delhi": "Delhi",
    "Bangalore": "Karnataka",
    "Hyderabad": "Telangana",
    "Ahmedabad": "Gujarat",
    "Chennai": "Tamil Nadu",
    "Kolkata": "West Bengal",
    "Pune": "Maharashtra",
    "Jaipur": "Rajasthan",
    "Surat": "Gujarat",
    "Lucknow": "Uttar Pradesh",
    "Kanpur": "Uttar Pradesh",
    "Nagpur": "Maharashtra",
    "Indore": "Madhya Pradesh",
    "Thane": "Maharashtra",
    "Bhopal": "Madhya Pradesh",
    "Visakhapatnam": "Andhra Pradesh",
    "Pimpri-Chinchwad": "Maharashtra",
    "Patna": "Bihar",
    "Vadodara": "Gujarat",
    "Ghaziabad": "Uttar Pradesh",
    "Ludhiana": "Punjab",
    "Agra": "Uttar Pradesh",
    "Nashik": "Maharashtra",
    "Faridabad": "Haryana",
    "Meerut": "Uttar Pradesh",
    "Rajkot": "Gujarat",
    "Kalyan-Dombivli": "Maharashtra",
    "Vasai-Virar": "Maharashtra",
    "Varanasi": "Uttar Pradesh",
    "Srinagar": "Jammu and Kashmir",
    "Aurangabad": "Maharashtra",
    "Dhanbad": "Jharkhand",
    "Amritsar": "Punjab",
    "Navi Mumbai": "Maharashtra",
    "Prayagraj": "Uttar Pradesh",
    "Howrah": "West Bengal",
    "Ranchi": "Jharkhand",
    "Jabalpur": "Madhya Pradesh",
    "Gwalior": "Madhya Pradesh",
    "Coimbatore": "Tamil Nadu",
    "Vijayawada": "Andhra Pradesh",
    "Jodhpur": "Rajasthan",
    "Madurai": "Tamil Nadu",
    "Raipur": "Chhattisgarh",
    "Kota": "Rajasthan",
    "Guwahati": "Assam",
    "Chandigarh": "Chandigarh",
    "Solapur": "Maharashtra",
    "Hubli-Dharwad": "Karnataka",
    "Tiruchirappalli": "Tamil Nadu",
    "Bareilly": "Uttar Pradesh",
    "Mysore": "Karnataka",
    "Tiruppur": "Tamil Nadu",
    "Gurgaon": "Haryana",
    "Aligarh": "Uttar Pradesh",
    "Jalandhar": "Punjab",
    "Bhubaneswar": "Odisha",
    "Salem": "Tamil Nadu",
    "Mira-Bhayandar": "Maharashtra",
    "Warangal": "Telangana",
    "Thiruvananthapuram": "Kerala",
    "Guntur": "Andhra Pradesh",
    "Bhiwandi": "Maharashtra",
    "Saharanpur": "Uttar Pradesh",
    "Gorakhpur": "Uttar Pradesh",
    "Bikaner": "Rajasthan",
    "Amravati": "Maharashtra",
    "Noida": "Uttar Pradesh",
    "Jamshedpur": "Jharkhand",
    "Bhilai": "Chhattisgarh",
    "Cuttack": "Odisha",
    "Firozabad": "Uttar Pradesh",
    "Kochi": "Kerala",
    "Nellore": "Andhra Pradesh",
    "Bhavnagar": "Gujarat",
    "Dehradun": "Uttarakhand",
    "Durgapur": "West Bengal",
    "Asansol": "West Bengal",
    "Nanded": "Maharashtra",
    "Kolhapur": "Maharashtra",
    "Ajmer": "Rajasthan",
    "Gulbarga": "Karnataka",
    "Jamnagar": "Gujarat",
    "Ujjain": "Madhya Pradesh",
    "Loni": "Uttar Pradesh",
    "Siliguri": "West Bengal",
    "Jhansi": "Uttar Pradesh",
    "Ulhasnagar": "Maharashtra",
    "Jammu": "Jammu and Kashmir",
    "Sangli-Miraj & Kupwad": "Maharashtra",
    "Mangalore": "Karnataka",
    "Erode": "Tamil Nadu",
    "Belgaum": "Karnataka",
    "Ambattur": "Tamil Nadu",
    "Tirunelveli": "Tamil Nadu",
    "Malegaon": "Maharashtra",
    "Gaya": "Bihar",
    "Jalgaon": "Maharashtra",
    "Udaipur": "Rajasthan",
    "Maheshtala": "West Bengal",
    "Davanagere": "Karnataka",
    "Kozhikode": "Kerala",
    "Kurnool": "Andhra Pradesh",
    "Akola": "Maharashtra",
    "Rajpur Sonarpur": "West Bengal",
    "Rajahmundry": "Andhra Pradesh",
    "Bokaro": "Jharkhand",
    "South Dumdum": "West Bengal",
    "Bellary": "Karnataka",
    "Patiala": "Punjab",
    "Gopalpur": "Odisha",
    "Agartala": "Tripura",
    "Bhagalpur": "Bihar",
    "Muzaffarnagar": "Uttar Pradesh",
    "Bhatpara": "West Bengal",
    "Panihati": "West Bengal",
    "Latur": "Maharashtra",
    "Dhule": "Maharashtra",
    "Tirupati": "Andhra Pradesh",
    "Rohtak": "Haryana",
    "Korba": "Chhattisgarh",
    "Bhilwara": "Rajasthan",
    "Berhampur": "Odisha",
    "Muzaffarpur": "Bihar",
    "Ahmednagar": "Maharashtra",
    "Mathura": "Uttar Pradesh",
    "Kollam": "Kerala",
    "Avadi": "Tamil Nadu",
    "Kadapa": "Andhra Pradesh",
    "Kamarhati": "West Bengal",
    "Bilaspur": "Chhattisgarh",
    "Shahjahanpur": "Uttar Pradesh",
    "Bijapur": "Karnataka",
    "Rampur": "Uttar Pradesh",
    "Shivamogga": "Karnataka",
    "Chandrapur": "Maharashtra",
    "Junagadh": "Gujarat",
    "Thrissur": "Kerala",
    "Alwar": "Rajasthan",
    "Bardhaman": "West Bengal",
    "Kulti": "West Bengal",
    "Kakinada": "Andhra Pradesh",
    "Nizamabad": "Telangana",
    "Parbhani": "Maharashtra",
    "Tumkur": "Karnataka",
    "Khammam": "Telangana",
    "Ozhukarai": "Puducherry",
    "Bihar Sharif": "Bihar",
    "Panipat": "Haryana",
    "Darbhanga": "Bihar",
    "Bally": "West Bengal",
    "Aizawl": "Mizoram",
    "Dewas": "Madhya Pradesh",
    "Ichalkaranji": "Maharashtra",
    "Karnal": "Haryana",
    "Bathinda": "Punjab",
    "Jalna": "Maharashtra",
    "Eluru": "Andhra Pradesh",
    "Kirari Suleman Nagar": "Delhi",
    "Barasat": "West Bengal",
    "Purnia": "Bihar",
    "Satna": "Madhya Pradesh",
    "Mau": "Uttar Pradesh",
    "Sonipat": "Haryana",
    "Farrukhabad": "Uttar Pradesh",
    "Sagar": "Madhya Pradesh",
    "Rourkela": "Odisha",
    "Durg": "Chhattisgarh",
    "Imphal": "Manipur",
    "Ratlam": "Madhya Pradesh",
    "Hapur": "Uttar Pradesh",
    "Arrah": "Bihar",
    "Karimnagar": "Telangana",
    "Anantapur": "Andhra Pradesh",
    "Etawah": "Uttar Pradesh",
    "Ambarnath": "Maharashtra",
    "North Dumdum": "West Bengal",
    "Bharatpur": "Rajasthan",
    "Begusarai": "Bihar",
    "New Delhi": "Delhi",
    "Gandhidham": "Gujarat",
    "Baranagar": "West Bengal",
    "Tiruvottiyur": "Tamil Nadu",
    "Puducherry": "Puducherry",
    "Sikar": "Rajasthan",
    "Thoothukudi": "Tamil Nadu",
    "Rewa": "Madhya Pradesh",
    "Mirzapur": "Uttar Pradesh",
    "Raichur": "Karnataka",
    "Pali": "Rajasthan"
}
# (Paste the above dictionary here)

cities = list(city_state_dict.keys())
states = list(set(city_state_dict.values()))




fake = Faker('en_IN')
random.seed(42)
Faker.seed(42)

NUM_USERS = 2000
NUM_DEVICES = 1000
NUM_RECIPIENTS = 1500
NUM_TRANSACTIONS = 20000



transaction_statuses = ["Completed", "Pending", "Failed"]

def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

# USERS TABLE
# Prepare the list of cities from the city_state_dict
cities = list(city_state_dict.keys())

user_rows, user_ids, user_city_map = [], [], {}
for i in range(1, NUM_USERS + 1):
    user_id = str(i)
    default_city = random.choice(cities)
    default_state = city_state_dict[default_city]

    user_rows.append((user_id, default_city, default_state))
    user_ids.append(user_id)
    # Sample 3 known cities for the user from the same city list
    user_city_map[user_id] = set(random.sample(cities, min(3, len(cities))))
user_schema = StructType([
    StructField("user_id", StringType(), False),
    StructField("default_city", StringType(), True),
    StructField("default_state", StringType(), True)
])
users_df = spark.createDataFrame(user_rows, schema=user_schema)
users_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("upi_fraud_schema.users")

# DEVICES TABLE
device_rows, device_ids, user_to_devices = [], [], defaultdict(list)
for i in range(1, NUM_DEVICES + 1):
    device_id = str(i)
    os_types = ['Android', 'iOS', 'Windows', 'Other']
    weights = [60.7, 25.0, 11.18, 1.12]
    norm_weights = [w / sum(weights) for w in weights]
    device_os = random.choices(os_types, weights=norm_weights)[0]

    user_id = random.choice(user_ids)
    device_rows.append((device_id, device_os))
    device_ids.append(device_id)
    user_to_devices[user_id].append(device_id)
device_schema = StructType([
    StructField("device_id", StringType(), False),
    StructField("device_os", StringType(), True)
])
devices_df = spark.createDataFrame(device_rows, schema=device_schema)
devices_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("upi_fraud_schema.devices")

# RECIPIENTS TABLE (Merchants)
recipient_rows, merchant_ids = [], []
for i in range(1, NUM_RECIPIENTS + 1):
    merchant_id = str(i)

    # Proper distribution for merchant categories
    merchant_categories = ['Utilities', 'Shopping', 'Food', 'Travel', 'Education', 'Healthcare']
    weights = [35, 25, 15, 10, 8, 7]
    norm_weights = [w / sum(weights) for w in weights]
    merchant_category = random.choices(merchant_categories, weights=norm_weights)[0]

    recipient_rows.append((merchant_id, merchant_category))
    merchant_ids.append(merchant_id)
recipient_schema = StructType([
    StructField("merchant_id", StringType(), False),
    StructField("merchant_category", StringType(), True)
])
recipients_df = spark.createDataFrame(recipient_rows, schema=recipient_schema)
recipients_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("upi_fraud_schema.recipients")

# TRANSACTIONS TABLE
transaction_rows = []
for i in range(1, NUM_TRANSACTIONS + 1):
    transaction_id = str(i)
    # Probabilities constraint on days 
    # Step 1: Define probabilities
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    weights = [20, 10, 9, 11,18 , 19, 13 ]
    norm_weights = [w / sum(weights) for w in weights]

    # Step 2: Pick a day of week
    chosen_day = random.choices(days, weights=norm_weights)[0]

    # Step 3: Find all dates in range that match chosen day
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2024, 6, 30)
    matching_dates = []
    current = start_date
    while current <= end_date:
        if current.strftime('%A') == chosen_day:
            matching_dates.append(current)
        current += timedelta(days=1)

    # Step 4: Pick a random date from those
    date_obj = random.choice(matching_dates)
    transaction_date = date_obj.date()

    # Step 5: Generate random time
    transaction_time = fake.time(pattern="%I:%M:%S %p")

    user_id = random.choice(user_ids)
    merchant_id = random.choice(merchant_ids)
    possible_devices = user_to_devices[user_id]
    device_id = random.choice(possible_devices) if possible_devices else random.choice(device_ids)
    # Randomise the UPI channels data  
    upi_channels = ["PhonePe", "GPay", "PayTM", "BHIM"]
    weights_upi = [0.52, 0.25, 0.15, 0.08]  # These represent the probabilities
    upi_channel = random.choices(upi_channels, weights=weights_upi)[0]

    transaction_city = random.choice(list(city_state_dict.keys()))
    transaction_state = city_state_dict[transaction_city]
    ip_address = fake.ipv4()
    transaction_status = random.choice(transaction_statuses)
    
    def get_realistic_amount(category):
        r = random.random()
        if category == "Education" or category == "Healthcare":
            if r < 0.75:
                return round(random.uniform(1000, 50000), 2)
            elif r < 0.95:
                return round(random.uniform(50001, 200000), 2)
            else:
                return round(random.uniform(200001, 500000), 2)  # Rare, high-value
        elif category == "Travel":
            if r < 0.85:
                return round(random.uniform(500, 20000), 2)
            else:
                return round(random.uniform(20001, 100000), 2)
        elif category == "Shopping":
            if r < 0.9:
                return round(random.uniform(100, 10000), 2)
            else:
                return round(random.uniform(10001, 100000), 2)
        elif category == "Utilities":
            if r < 0.95:
                return round(random.uniform(50, 2000), 2)
            else:
                return round(random.uniform(2001, 10000), 2)
        elif category == "Food":
            if r < 0.98:
                return round(random.uniform(50, 2000), 2)
            else:
                return round(random.uniform(2001, 10000), 2)
        else:
            return round(random.uniform(1, 5000), 2)
    
    amount = get_realistic_amount(merchant_category)
    transaction_amount_deviation = round(random.uniform(-100, 100), 2)
    # --- Improved Risk Score Calculation ---
    risk_score = 0

    # Category-aware high amount thresholds (add this dict at the top of your script)
    category_thresholds = {
        "Utilities": 2000,
        "Shopping": 10000,
        "Food": 2000,
        "Travel": 20000,
        "Education": 200000,
        "Healthcare": 200000
    }
    high_amount_threshold = category_thresholds.get(merchant_category, 5000)

    # High amount buckets (now category-aware)
    if amount > high_amount_threshold:
        risk_score += 40
    elif amount > high_amount_threshold * 0.6:
        risk_score += 25
    elif amount > high_amount_threshold * 0.2:
        risk_score += 10

    # Large deviation from user's normal transaction amount (relative if possible)
    if 'user_avg_amount' in locals() and user_avg_amount > 0:
        deviation_pct = abs(amount - user_avg_amount) / user_avg_amount
        if deviation_pct > 1.0:
            risk_score += 25
        elif deviation_pct > 0.5:
            risk_score += 10
    else:
        if abs(transaction_amount_deviation) > 100:
            risk_score += 25
        elif abs(transaction_amount_deviation) > 50:
            risk_score += 10

    # Location anomaly: city or state not in user's known set
    if transaction_city not in user_city_map[user_id]:
        risk_score += 15
    if transaction_state not in [city_state_dict[city] for city in user_city_map[user_id]]:
        risk_score += 10

    # Device anomaly: device rarely used by this user
    if device_id not in user_to_devices[user_id][:2]:  # not among user's 2 most common devices
        risk_score += 10

    # Odd hour: between 12 AM and 5 AM (fix for 12 AM edge case)
    from datetime import datetime
    txn_hour = datetime.strptime(transaction_time, "%I:%M:%S %p").hour
    if 0 <= txn_hour < 5:
        risk_score += 15

    # Status and amount interaction
    if transaction_status == 'Failed' and amount > 3000:
        risk_score += 20
    elif transaction_status == 'Pending' and amount > 3000:
        risk_score += 10


    # Normalize and assign fraud label
    risk_score = min(risk_score / 100, 1.0)
    fraud = 1 if risk_score > 0.7 else 0


    transaction_rows.append((
        transaction_id, transaction_date, transaction_time, user_id, merchant_id, device_id, upi_channel,
        transaction_city, transaction_state, ip_address, transaction_status, amount, transaction_amount_deviation, fraud
    ))

transaction_schema = StructType([
    StructField("transaction_id", StringType(), False),
    StructField("transaction_date", DateType(), True),
    StructField("transaction_time", StringType(), True),
    StructField("user_id", StringType(), False),
    StructField("merchant_id", StringType(), False),
    StructField("device_id", StringType(), False),
    StructField("upi_channel", StringType(), True),
    StructField("transaction_city", StringType(), True),
    StructField("transaction_state", StringType(), True),
    StructField("ip_address", StringType(), True),
    StructField("transaction_status", StringType(), True),
    StructField("amount", FloatType(), True),
    StructField("transaction_amount_deviation", FloatType(), True),
    StructField("fraud", IntegerType(), True)
])
transactions_df = spark.createDataFrame(transaction_rows, schema=transaction_schema)
transactions_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("upi_fraud_schema.transactions")


#####Adding joins on tables to create "Features" Table

In [0]:
from pyspark.sql.functions import hour, unix_timestamp, when, date_format, col

# Read tables
users_df = spark.table("upi_fraud_schema.users")
devices_df = spark.table("upi_fraud_schema.devices")
recipients_df = spark.table("upi_fraud_schema.recipients")
transactions_df = spark.table("upi_fraud_schema.transactions")

# Join and engineer features
features_df = transactions_df.alias("t") \
    .join(users_df.alias("u"), col("t.user_id") == col("u.user_id")) \
    .join(devices_df.alias("d"), col("t.device_id") == col("d.device_id")) \
    .join(recipients_df.alias("r"), col("t.merchant_id") == col("r.merchant_id")) \
    .withColumn("hour", hour(unix_timestamp(col("t.transaction_time"), "hh:mm:ss a").cast("timestamp"))) \
    .withColumn("day_of_week", date_format(col("t.transaction_date"), "EEEE")) \
    .withColumn("is_high_value", (col("t.amount") > 3000).cast("int")) \
    .withColumn("is_odd_hour", when((col("hour") < 6) | (col("hour") > 22), 1).otherwise(0)) \
    .select(
        "t.transaction_id", "t.user_id", "t.merchant_id", "t.device_id",
        "t.amount", "hour", "day_of_week",
        "t.transaction_amount_deviation", "is_high_value", "is_odd_hour",
        "t.upi_channel", "t.transaction_status",
        "r.merchant_category", "d.device_os", "u.default_city", "u.default_state",
        "t.transaction_city", "t.transaction_state", "t.fraud",
        "t.transaction_date", "t.transaction_time"
    )

features_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("upi_fraud_schema.features")


In [0]:
%sql
SELECT * FROM upi_fraud_schema.features LIMIT 10;


transaction_id,user_id,merchant_id,device_id,amount,hour,day_of_week,transaction_amount_deviation,is_high_value,is_odd_hour,upi_channel,transaction_status,merchant_category,device_os,default_city,default_state,transaction_city,transaction_state,fraud,transaction_date,transaction_time
1,1614,889,389,293.01,18,Saturday,-78.0,0,0,PhonePe,Completed,Food,Android,Jaipur,Rajasthan,Ichalkaranji,Maharashtra,0,2023-10-14,06:40:37 PM
2,675,1312,855,1606.29,18,Friday,-45.27,0,0,PhonePe,Pending,Utilities,iOS,Ujjain,Madhya Pradesh,Loni,Uttar Pradesh,0,2023-03-03,06:46:35 PM
3,776,216,802,647.77,0,Monday,64.27,0,1,BHIM,Failed,Utilities,Android,Rajahmundry,Andhra Pradesh,Raipur,Chhattisgarh,0,2023-07-17,12:43:03 AM
4,715,1435,586,1798.31,21,Thursday,-65.85,0,0,PayTM,Pending,Utilities,Windows,Mysore,Karnataka,Kulti,West Bengal,0,2024-04-18,09:59:19 PM
5,371,707,826,1706.78,2,Sunday,-4.63,0,1,GPay,Failed,Utilities,Android,Alwar,Rajasthan,Chandrapur,Maharashtra,1,2023-12-10,02:22:35 AM
6,1407,1052,788,571.37,11,Monday,83.69,0,0,GPay,Pending,Utilities,iOS,Etawah,Uttar Pradesh,Howrah,West Bengal,0,2023-01-09,11:51:40 AM
7,1449,511,345,8387.95,16,Friday,0.45,1,0,GPay,Pending,Shopping,Android,Thiruvananthapuram,Kerala,Guwahati,Assam,1,2023-12-01,04:10:50 PM
8,1067,1472,903,219.54,23,Tuesday,-38.87,0,1,BHIM,Pending,Utilities,Android,Jalna,Maharashtra,Salem,Tamil Nadu,0,2024-01-16,11:22:49 PM
9,1300,163,79,214.94,13,Saturday,-42.83,0,0,GPay,Completed,Travel,Other,Jabalpur,Madhya Pradesh,Nashik,Maharashtra,0,2023-03-25,01:56:43 PM
10,1971,869,639,9104.76,19,Sunday,39.58,1,0,PhonePe,Failed,Shopping,Android,Gurgaon,Haryana,Bilaspur,Chhattisgarh,1,2023-01-08,07:34:53 PM


Saving df as csv file

In [0]:
import pandas as pd
df=features_df
pdf=features_df.toPandas()


pdf.to_csv('upi_transactions.csv', index=False)
print("Dataset saved as 'upi_transactions_2024.csv'")

Dataset saved as 'upi_transactions_2024.csv'


######IMPORTING LIBRARIES


In [0]:
# importing libraries
# %pip install pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as px
import seaborn as sns

View Schema of the df

In [0]:
df.columns
# df.printSchema

['transaction_id',
 'user_id',
 'merchant_id',
 'device_id',
 'amount',
 'hour',
 'day_of_week',
 'transaction_amount_deviation',
 'is_high_value',
 'is_odd_hour',
 'upi_channel',
 'transaction_status',
 'merchant_category',
 'device_os',
 'default_city',
 'default_state',
 'transaction_city',
 'transaction_state',
 'fraud',
 'transaction_date',
 'transaction_time']

# Data Preparation
### -duplicate, missing, unique, removing or dropping unique values

Data preparation is a crucial step in the data analysis and machine learning pipeline. It involves cleaning, transforming, and organizing raw data into a format suitable for analysis. Key tasks in data preparation include:

- Cleaning Data: Removing duplicates, handling missing values, and correcting errors to ensure data quality.
- Transforming Data: Standardizing data types, normalizing values, and creating new features to enhance the dataset.
- Organizing Data: Structuring data into tables or schemas that facilitate efficient querying and analysis.

Effective data preparation improves the accuracy and reliability of the analysis and models built on the data

##### DUPLICATE VALUES


In [0]:
from pyspark.sql.functions import count

# Find all rows that are duplicated across all columns
df.groupBy(df.columns) \
    .count() \
    .filter("count > 1") \
    .show(truncate=False)
    #  .count()


# df.duplictaed()->pandas


+--------------+-------+-----------+---------+------+----+-----------+----------------------------+-------------+-----------+-----------+------------------+-----------------+---------+------------+-------------+----------------+-----------------+-----+----------------+----------------+-----+
|transaction_id|user_id|merchant_id|device_id|amount|hour|day_of_week|transaction_amount_deviation|is_high_value|is_odd_hour|upi_channel|transaction_status|merchant_category|device_os|default_city|default_state|transaction_city|transaction_state|fraud|transaction_date|transaction_time|count|
+--------------+-------+-----------+---------+------+----+-----------+----------------------------+-------------+-----------+-----------+------------------+-----------------+---------+------------+-------------+----------------+-----------------+-----+----------------+----------------+-----+
+--------------+-------+-----------+---------+------+----+-----------+----------------------------+-------------+--------

##### MISSING VALUES

In [0]:

from pyspark.sql.functions import col, sum as spark_sum
from functools import reduce

# This will show you how many nulls are present in each column:
missing_counts = df.select([
    spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns
])
missing_counts.show()


# This will display all rows that have at least one null value
df.filter(
    reduce(lambda a, b: a | b, (col(c).isNull() for c in df.columns))
).show()


# This gives you the number of rows with at least one null:
num_missing_rows = df.filter(
    reduce(lambda a, b: a | b, (col(c).isNull() for c in df.columns))
).count()
print("Rows with any missing value:", num_missing_rows)

+--------------+-------+-----------+---------+------+----+-----------+----------------------------+-------------+-----------+-----------+------------------+-----------------+---------+------------+-------------+----------------+-----------------+-----+----------------+----------------+
|transaction_id|user_id|merchant_id|device_id|amount|hour|day_of_week|transaction_amount_deviation|is_high_value|is_odd_hour|upi_channel|transaction_status|merchant_category|device_os|default_city|default_state|transaction_city|transaction_state|fraud|transaction_date|transaction_time|
+--------------+-------+-----------+---------+------+----+-----------+----------------------------+-------------+-----------+-----------+------------------+-----------------+---------+------------+-------------+----------------+-----------------+-----+----------------+----------------+
|             0|      0|          0|        0|     0|   0|          0|                           0|            0|          0|          0|  

How to deal with missing values if any
-drop the missing values ( only if proportion is very less)
-fill th emissing values
-froward fill
-back fill
-linear regression
-mean values (but it is sensitive to outliers)
-median values ( not sensitive to outliers)

In [0]:
num_rows = df.count()
num_cols = len(df.columns)
print(f"Shape: ({num_rows}, {num_cols})")


Shape: (20000, 21)


### DEALING WITH UNIQUES VALUES


In [0]:
from pyspark.sql.functions import countDistinct
import pandas as pd

# Compute unique counts for each column
unique_counts = [(c, df.select(countDistinct(c)).first()[0]) for c in df.columns]

# Convert to Spark DataFrame for display
unique_counts_df = spark.createDataFrame(unique_counts, ["column_name", "unique_count"])
unique_counts_df.show(truncate=False)


+----------------------------+------------+
|column_name                 |unique_count|
+----------------------------+------------+
|transaction_id              |20000       |
|user_id                     |2000        |
|merchant_id                 |1500        |
|device_id                   |1000        |
|amount                      |19079       |
|hour                        |24          |
|day_of_week                 |7           |
|transaction_amount_deviation|12603       |
|is_high_value               |2           |
|is_odd_hour                 |2           |
|upi_channel                 |4           |
|transaction_status          |3           |
|merchant_category           |6           |
|device_os                   |4           |
|default_city                |191         |
|default_state               |26          |
|transaction_city            |191         |
|transaction_state           |26          |
|fraud                       |2           |
|transaction_date            |54

### Drop the unique values from the data



In [0]:

df.show(5, truncate=False)

+--------------+-------+-----------+---------+-------+----+-----------+----------------------------+-------------+-----------+-----------+------------------+-----------------+---------+------------+--------------+----------------+-----------------+-----+----------------+----------------+
|transaction_id|user_id|merchant_id|device_id|amount |hour|day_of_week|transaction_amount_deviation|is_high_value|is_odd_hour|upi_channel|transaction_status|merchant_category|device_os|default_city|default_state |transaction_city|transaction_state|fraud|transaction_date|transaction_time|
+--------------+-------+-----------+---------+-------+----+-----------+----------------------------+-------------+-----------+-----------+------------------+-----------------+---------+------------+--------------+----------------+-----------------+-----+----------------+----------------+
|1             |1614   |889        |389      |293.01 |18  |Saturday   |-78.0                       |0            |0          |PhonePe

### Analysing the classification dataset

In [0]:
from pyspark.sql.functions import col

# For fraud transactions (fraud = 1)
fraud_df = df.filter(col('fraud') == 1)

# For legitimate transactions (fraud = 0)
normal_df = df.filter(col('fraud') == 0)

print(f"Fraud cases: {fraud_df.count()}")
print(f"Legitimate cases: {normal_df.count()}")


Fraud cases: 2064
Legitimate cases: 17936


# Machine Learning Models:


###Label Encoding & Scaling

##### Label Encoding
- Label encoding is a technique used to convert categorical text data into numerical data. Each unique category value is assigned an integer value. This is useful for machine learning algorithms that require numerical input. However, it can introduce ordinal relationships between categories that may not exist.

##### Scaling
- Scaling is the process of transforming data to fit within a specific range, typically 0 to 1 or -1 to 1. This is important for algorithms that are sensitive to the scale of data, such as gradient descent-based algorithms. Common scaling techniques include Min-Max Scaling and Standardization (Z-score normalization).

In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Assume pdf is your pandas DataFrame

# Categorical columns to encode
categorical_cols = [
    # 'user_id',
    # 'merchant_id',
    # 'device_id',
    'day_of_week',
    'upi_channel',
    'transaction_status',
    'merchant_category',
    'device_os',
    # 'default_city',
    # 'default_state',
    'transaction_city',
    'transaction_state'
]

# Numerical columns to scale
numeric_cols = [
    'amount',
    'hour',
    'transaction_amount_deviation',
    'is_high_value',
    'is_odd_hour'
]

# 2. Label Encoding for Categorical Columns
for col in categorical_cols:
    le = LabelEncoder()
    pdf[col + '_enc'] = le.fit_transform(pdf[col].astype(str))


# 3. Standard Scaling for Numerical Columns
scaler = StandardScaler()
pdf[numeric_cols] = scaler.fit_transform(pdf[numeric_cols])

# 4. Prepare final feature list for modeling
feature_cols = [col + '_enc' for col in categorical_cols] + numeric_cols

# 5. X and y for modeling
X = pdf[feature_cols]
y = pdf['fraud']

# (Optional) Show the first few rows of the processed DataFrame
print(pdf[feature_cols + ['fraud']].head())
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pickle

# Categorical and numeric columns
categorical_cols = [
    'day_of_week',
    'upi_channel',
    'transaction_status',
    'merchant_category',
    'device_os',
    'transaction_city',
    'transaction_state'
]

numeric_cols = [
    'amount',
    'hour',
    'transaction_amount_deviation',
    'is_high_value',
    'is_odd_hour'
]

# 1. Dictionary to store encoders
label_encoders = {}

# 2. Fit and apply label encoding, store encoders
for col in categorical_cols:
    le = LabelEncoder()
    pdf[col + '_enc'] = le.fit_transform(pdf[col].astype(str))
    label_encoders[col] = le

# 3. Fit and apply standard scaling
scaler = StandardScaler()
pdf[numeric_cols] = scaler.fit_transform(pdf[numeric_cols])

# 4. Final features and labels
feature_cols = [col + '_enc' for col in categorical_cols] + numeric_cols
X = pdf[feature_cols]
y = pdf['fraud']

# Optional: Display few rows
print(pdf[feature_cols + ['fraud']].head())

# 5. Save encoder dictionary and scaler
with open("label_encoders.pkl", "wb") as f:
    pickle.dump(label_encoders, f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)


   day_of_week_enc  upi_channel_enc  ...  is_odd_hour  fraud
0                2                3  ...    -0.643527      0
1                0                3  ...     1.553936      0
2                1                3  ...    -0.643527      0
3                1                1  ...    -0.643527      0
4                2                3  ...    -0.643527      0

[5 rows x 13 columns]
   day_of_week_enc  upi_channel_enc  ...  is_odd_hour  fraud
0                2                3  ...    -0.643527      0
1                0                3  ...     1.553936      0
2                1                3  ...    -0.643527      0
3                1                1  ...    -0.643527      0
4                2                3  ...    -0.643527      0

[5 rows x 13 columns]


# Train-Test Split

Train-test split is a technique used to evaluate the performance of a machine learning model. It involves dividing the dataset into two subsets:

- Training Set: Used to train the model.
- Test Set: Used to evaluate the model's performance on unseen data.

This helps in assessing how well the model generalizes to new data. A common practice is to allocate 70-80% of the data for training and the remaining 20-30% for testing.

In [0]:
from sklearn.model_selection import train_test_split

# X = your features DataFrame
# y = your target variable (e.g., pdf['fraud'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,         # 30% for testing, 70% for training
    random_state=42,       # for reproducibility
    stratify=y             # keeps the fraud ratio similar in both sets
)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)


Train shape: (14000, 12) (14000,)
Test shape: (6000, 12) (6000,)


# Random Forest Classification

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# 1. Initialize the Random Forest model
clf = RandomForestClassifier(
    n_estimators=100,        # Number of trees
    random_state=42,         # For reproducibility
    class_weight='balanced'  # Helps with class imbalance
)

# 2. Train the model
clf.fit(X_train, y_train)

# 3. Make predictions
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]  # Probability of class 1 (fraud)

# 4. Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))
print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")


Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.98      5381
           1       0.83      0.91      0.87       619

    accuracy                           0.97      6000
   macro avg       0.91      0.94      0.93      6000
weighted avg       0.97      0.97      0.97      6000

ROC AUC Score: 0.9925


Summary:
The model shows excellent performance, with 97% overall accuracy and a high ROC AUC of 0.9925. It detects 89% of fraud cases (recall) with 84% precision, while non-fraud detection is near perfect. The model is highly effective at distinguishing fraudulent from non-fraudulent transactions, with only a small number of false positives for fraud.

In [0]:
import pickle

# Save the trained model to a file
with open('randomforest.pkl', 'wb') as file:
    pickle.dump(clf, file)

print("Model saved as randomforest.pkl")


Model saved as randomforest.pkl


## XGBoost model

In [0]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score


# Set scale_pos_weight to balance classes (optional, recommended for imbalanced data)
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

# Initialize and train XGBoost


model = xgb.XGBClassifier(
    objective='binary:logistic',
    scale_pos_weight=scale_pos_weight,
    eval_metric='auc',
    random_state=42
)

model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))


              precision    recall  f1-score   support

           0       0.99      0.97      0.98      5381
           1       0.80      0.94      0.86       619

    accuracy                           0.97      6000
   macro avg       0.90      0.95      0.92      6000
weighted avg       0.97      0.97      0.97      6000

ROC AUC Score: 0.9913607352381787


Summary:
The model achieves 97% accuracy and a ROC AUC of 0.99, indicating excellent discrimination between fraud and non-fraud. It detects 94% of fraud cases (recall) with 82% precision. Non-fraud detection remains near perfect. Overall, the model is highly effective for fraud detection, with strong recall for fraudulent transactions.

In [0]:
import pickle

# Save the trained model to a file
with open('xgb_model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model saved as xgb_model.pkl")


Model saved as xgb_model.pkl
